In [1]:
import aiohttp
import aiomoex
import pandas as pd
from datetime import datetime, timedelta

In [2]:
async def get_ticks(limit=5000, reversed=1):
    # request_url = "https://iss.moex.com/iss/engines/stock/" "markets/shares/boards/TQBR/securities.json"
    request_url = "https://iss.moex.com/iss/engines/stock/" "markets/shares/securities/SBER/trades.json"
    # arguments = {"securities.columns": ("SECID," "REGNUMBER," "LOTSIZE," "SHORTNAME")}

    async with aiohttp.ClientSession() as session:
        iss = aiomoex.ISSClient(session, request_url, {'reversed': reversed, 'limit': limit})
        data = await iss.get()
        df = pd.DataFrame(data["trades"])
        # df.set_index("SECID", inplace=True)
    return df

In [3]:
df_ticks = await get_ticks()
display(df_ticks)

,TRADENO,TRADETIME,BOARDID,SECID,PRICE,QUANTITY,VALUE,PERIOD,TRADETIME_GRP,SYSTIME,BUYSELL,DECIMALS,TRADINGSESSION,TRADEDATE,TRADE_SESSION_DATE
0,12649098631,17:18:47,TQBR,SBER,318.00,160,508800.00,N,1718,2025-03-12 17:18:48,S,2,1,2025-03-12,2025-03-12
1,12649097804,17:18:46,SMAL,SBER,319.99,2,639.98,N,1718,2025-03-12 17:18:46,B,2,1,2025-03-12,2025-03-12
2,12649097546,17:18:46,TQBR,SBER,318.02,6,19081.20,N,1718,2025-03-12 17:18:46,B,2,1,2025-03-12,2025-03-12
3,12649097334,17:18:45,TQBR,SBER,318.00,445,1415100.00,N,1718,2025-03-12 17:18:46,S,2,1,2025-03-12,2025-03-12
4,12649097333,17:18:45,TQBR,SBER,318.00,100,318000.00,N,1718,2025-03-12 17:18:46,S,2,1,2025-03-12,2025-03-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,12648763651,16:32:38,TQBR,SBER,319.40,1,3194.00,N,1632,2025-03-12 16:32:38,S,2,1,2025-03-12,2025-03-12
4996,12648763637,16:32:38,TQBR,SBER,319.41,1,3194.10,N,1632,2025-03-12 16:32:38,B,2,1,2025-03-12,2025-03-12
4997,12648763620,16:32:37,TQBR,SBER,319.40,14,44716.00,N,1632,2025-03-12 16:32:37,S,2,1,2025-03-12,2025-03-12
4998,12648763225,16:32:32,TQBR,SBER,319.40,50,159700.00,N,1632,2025-03-12 16:32:32,S,2,1,2025-03-12,2025-03-12


In [4]:
async def fetch_moex_data(ticker, timeframe='m', days=1825):
    """
    Асинхронная загрузка свечных данных с Московской биржи через aiomoex.
    """
    # Определяем конечную и начальную даты
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)

    # Приводим к строковому формату YYYY-MM-DD
    start_str = start_date.strftime('%Y-%m-%d')
    end_str = end_date.strftime('%Y-%m-%d')

    # Маппинг интервалов
    interval_map = {'D': 24, 'W': 7, 'H': 60, 'M': 31, 'm': 1}
    interval = interval_map.get(timeframe, 1)  # По умолчанию минутные свечи

    print(f"📊 Загружаем данные для {ticker} с {start_str} по {end_str} (таймфрейм: {timeframe})")

    async with aiohttp.ClientSession() as session:
        # Запрашиваем свечные данные
        data = await aiomoex.get_market_candles(
            session,
            security=ticker,
            interval=interval,
            start=start_str,
            end=end_str
        )

    # Проверяем, есть ли данные
    if not data:
        raise ValueError(f"❌ Не удалось получить данные для {ticker}")

    # Создаем DataFrame
    df = pd.DataFrame(data)

    # Переименовываем колонки
    column_mapping = {
        'begin': 'date',
        'open': 'open',
        'high': 'high',
        'low': 'low',
        'close': 'close',
        'value': 'volume' # если volume, то объём торгов в штуках
    }
    df = df.rename(columns=column_mapping)

    # Преобразуем дату в datetime и ставим индекс
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)

    # Оставляем только нужные столбцы
    df = df[['open', 'high', 'low', 'close', 'volume']]

    print(f"✅ Загружено {len(df)} свечей для {ticker}")

    return df, start_str, end_str

In [5]:
df, start_str, end_str = await fetch_moex_data("SBER", timeframe='m5', days=5)
display(df)
print(start_str, ' to ', end_str)

📊 Загружаем данные для SBER с 2025-03-07 по 2025-03-12 (таймфрейм: m5)
✅ Загружено 3611 свечей для SBER


,open,high,low,close,volume,volume
date,,,,,,
2025-03-07 06:59:00,315.19,315.19,315.19,315.19,349860.9,1110
2025-03-07 07:00:00,315.30,315.63,315.19,315.48,2323864.0,7370
2025-03-07 07:01:00,315.49,315.50,315.49,315.49,4013152.2,12720
2025-03-07 07:02:00,315.49,315.50,315.49,315.50,2965666.9,9400
2025-03-07 07:03:00,315.50,315.50,315.41,315.50,1526961.1,4840
...,...,...,...,...,...,...
2025-03-12 17:29:00,318.50,318.68,318.50,318.68,5877851.9,18450
2025-03-12 17:30:00,318.67,318.84,318.67,318.77,3257979.1,10220
2025-03-12 17:31:00,318.76,318.88,318.65,318.69,4669873.1,14650


2025-03-07  to  2025-03-12
